In [ ]:
from fastai import *        # Quick access to most common functionality
from fastai.text import *   # Quick access to NLP functionality
from fastai.docs import *   # Access to example data provided with fastai

# Text example

An example of creating a language model and then transfering to a classifier.

In [ ]:
untar_data(IMDB_PATH)
IMDB_PATH

PosixPath('../data/imdb_sample')

Open and view the independent and dependent variables:

In [ ]:
df = pd.read_csv(IMDB_PATH/'train.csv', header=None)
df.head()

0                                                  1
0  0  Un-bleeping-believable! Meg Ryan doesn't even ...
1  1  This is a extremely well-made film. The acting...
2  0  Every once in a long while a movie will come a...
3  1  Name just says it all. I watched this movie wi...
4  0  This movie succeeds at being one of the most u...

In [ ]:
classes = read_classes(IMDB_PATH/'classes.txt')
classes[0], classes[1]

('negative', 'positive')

Create a `DataBunch` for each of the language model and the classifier:

In [ ]:
data_lm = text_data_from_csv(Path(IMDB_PATH), data_func=lm_data)
data_clas = text_data_from_csv(Path(IMDB_PATH), data_func=classifier_data, vocab=data_lm.train_ds.vocab)

[fast.ai](http://www.fast.ai/) has a pre-trained English model available that we can download.

In [ ]:
download_wt103_model()

We'll fine-tune the language model:

In [ ]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

Total time: 00:47
epoch  train loss  valid loss  accuracy
0      4.918286    4.157264    0.243858  (00:23)
1      4.644932    4.085907    0.253978  (00:23)



Save our language model's encoder:

In [ ]:
learn.save_encoder('enc')

Fine tune it to create a classifier:

In [ ]:
learn = RNNLearner.classifier(data_clas)
learn.load_encoder('enc')
learn.fit(3, 1e-3)

Total time: 00:48
epoch  train loss  valid loss  accuracy
0      0.693048    0.673415    0.615000  (00:16)
1      0.658173    0.629967    0.685000  (00:16)
2      0.646465    0.600343    0.695000  (00:16)

